In [ ]:
import earthchem
earthchem.__version__

Make a rest client

In [ ]:
q = earthchem.Query(author='klump')
q.url

How many records have we found?

In [ ]:
q.count()

In [ ]:
q.dataframe()

Let's try another author

In [ ]:
q = earthchem.Query(author='barnes')
q.count()

Get the first 50 records

In [ ]:
df = q.dataframe()
df.head()

Now we can generate a plot of this data

In [ ]:
%matplotlib inline

df.plot('longitude', 'latitude', 'scatter')

In [ ]:
df.plot('al2o3', 'mgo', 'scatter')